<a href="https://colab.research.google.com/github/Ashfaque-Al-Sinan/AI-Academic/blob/main/Copy_of_goal_based_vaccum_cleaner_1031.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import random
from enum import Enum
from typing import List, Tuple, Set
import time

class Action(Enum):
    MOVE_UP = "move_up"
    MOVE_DOWN = "move_down"
    MOVE_LEFT = "move_left"
    MOVE_RIGHT = "move_right"
    CLEAN = "clean"
    IDLE = "idle"

class CellState(Enum):
    CLEAN = 0
    DIRTY = 1
    OBSTACLE = 2

class CleaningAgent:
    def __init__(self, room_size: Tuple[int, int] = (5, 5), dirt_prob: float = 0.3):
        self.room_size = room_size
        self.position = (0, 0)  # Start at top-left
        self.performance = 0
        self.actions_taken = 0

        # Initialize room with random dirt and some obstacles
        self.room = self._initialize_room(dirt_prob)
        self.cleaned_cells = set()

        # Agent's internal model of the world
        self.model = {
            'room_state': [[CellState.CLEAN for _ in range(room_size[1])]
                          for _ in range(room_size[0])],
            'agent_position': self.position,
            'dirt_locations': set()
        }

        # Update model with initial perceptions
        self._update_model()

    def _initialize_room(self, dirt_prob: float) -> List[List[CellState]]:
        """Initialize the room with dirt and obstacles"""
        room = []
        for i in range(self.room_size[0]):
            row = []
            for j in range(self.room_size[1]):
                # 10% chance of obstacle, except starting position
                if (i, j) != self.position and random.random() < 0.1:
                    row.append(CellState.OBSTACLE)
                elif random.random() < dirt_prob:
                    row.append(CellState.DIRTY)
                else:
                    row.append(CellState.CLEAN)
            room.append(row)
        return room

    def _update_model(self):
        """Update the agent's internal model based on current perceptions"""
        for i in range(self.room_size[0]):
            for j in range(self.room_size[1]):
                if self.room[i][j] == CellState.DIRTY:
                    self.model['dirt_locations'].add((i, j))
                elif self.room[i][j] == CellState.CLEAN:
                    self.model['dirt_locations'].discard((i, j))

        self.model['agent_position'] = self.position

    def perceive(self) -> dict:
        """Get current perceptions from the environment"""
        x, y = self.position
        return {
            'current_cell_state': self.room[x][y],
            'agent_position': self.position,
            'adjacent_cells': self._get_adjacent_cells()
        }

    def _get_adjacent_cells(self) -> List[Tuple[int, int]]:
        """Get valid adjacent cells that are within bounds and not obstacles"""
        x, y = self.position
        adjacent = []

        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_x, new_y = x + dx, y + dy
            if (0 <= new_x < self.room_size[0] and
                0 <= new_y < self.room_size[1] and
                self.room[new_x][new_y] != CellState.OBSTACLE):
                adjacent.append((new_x, new_y))

        return adjacent

    def _goal_test(self) -> bool:
        """Check if cleaning goal is achieved"""
        for i in range(self.room_size[0]):
            for j in range(self.room_size[1]):
                if self.room[i][j] == CellState.DIRTY:
                    return False
        return True

    def _formulate_plan(self) -> List[Action]:
        """Formulate a plan to achieve the cleaning goal"""
        perceptions = self.perceive()

        # If current cell is dirty, clean it
        if perceptions['current_cell_state'] == CellState.DIRTY:
            return [Action.CLEAN]

        # Find the nearest dirty cell using BFS
        nearest_dirt = self._find_nearest_dirty_cell()

        if nearest_dirt:
            return self._plan_path_to_target(nearest_dirt)

        # No dirty cells found (goal achieved)
        return [Action.IDLE]

    def _find_nearest_dirty_cell(self) -> Tuple[int, int]:
        """Find the nearest dirty cell using BFS"""
        visited = set()
        queue = [(self.position, [])]  # (position, path)

        while queue:
            current_pos, path = queue.pop(0)

            if current_pos in visited:
                continue

            visited.add(current_pos)

            x, y = current_pos
            if self.room[x][y] == CellState.DIRTY:
                return current_pos

            # Add adjacent cells to queue
            for neighbor in self._get_valid_neighbors(current_pos):
                if neighbor not in visited:
                    queue.append((neighbor, path + [neighbor]))

        return None

    def _get_valid_neighbors(self, position: Tuple[int, int]) -> List[Tuple[int, int]]:
        """Get valid neighboring cells for a given position"""
        x, y = position
        neighbors = []

        for dx, dy in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            new_x, new_y = x + dx, y + dy
            if (0 <= new_x < self.room_size[0] and
                0 <= new_y < self.room_size[1] and
                self.room[new_x][new_y] != CellState.OBSTACLE):
                neighbors.append((new_x, new_y))

        return neighbors

    def _plan_path_to_target(self, target: Tuple[int, int]) -> List[Action]:
        """Plan a path from current position to target using BFS"""
        visited = set()
        queue = [(self.position, [])]  # (position, actions)

        while queue:
            current_pos, actions = queue.pop(0)

            if current_pos in visited:
                continue

            visited.add(current_pos)

            if current_pos == target:
                return actions

            # Try all possible moves
            for action, new_pos in self._get_possible_moves(current_pos):
                if new_pos not in visited:
                    queue.append((new_pos, actions + [action]))

        return [Action.IDLE]  # No path found

    def _get_possible_moves(self, position: Tuple[int, int]) -> List[Tuple[Action, Tuple[int, int]]]:
        """Get all possible moves from a given position"""
        x, y = position
        moves = []

        # Define movement patterns
        movement_patterns = [
            (Action.MOVE_UP, (x-1, y)),
            (Action.MOVE_DOWN, (x+1, y)),
            (Action.MOVE_LEFT, (x, y-1)),
            (Action.MOVE_RIGHT, (x, y+1))
        ]

        for action, new_pos in movement_patterns:
            new_x, new_y = new_pos
            if (0 <= new_x < self.room_size[0] and
                0 <= new_y < self.room_size[1] and
                self.room[new_x][new_y] != CellState.OBSTACLE):
                moves.append((action, new_pos))

        return moves

    def execute_action(self, action: Action):
        """Execute an action in the environment"""
        self.actions_taken += 1

        if action == Action.CLEAN:
            x, y = self.position
            if self.room[x][y] == CellState.DIRTY:
                self.room[x][y] = CellState.CLEAN
                self.cleaned_cells.add((x, y))
                self.performance += 10  # Reward for cleaning
                print(f"Cleaned cell {self.position}")
            else:
                self.performance -= 2  # Penalty for unnecessary cleaning

        elif action in [Action.MOVE_UP, Action.MOVE_DOWN, Action.MOVE_LEFT, Action.MOVE_RIGHT]:
            new_position = self._calculate_new_position(action)
            if self._is_valid_position(new_position):
                self.position = new_position
                self.performance -= 1  # Cost for moving
                print(f"Moved to {self.position}")
            else:
                self.performance -= 3  # Penalty for invalid move

        elif action == Action.IDLE:
            self.performance -= 1  # Small penalty for idling

        # Update the agent's model
        self._update_model()

    def _calculate_new_position(self, action: Action) -> Tuple[int, int]:
        """Calculate new position based on action"""
        x, y = self.position
        if action == Action.MOVE_UP:
            return (x-1, y)
        elif action == Action.MOVE_DOWN:
            return (x+1, y)
        elif action == Action.MOVE_LEFT:
            return (x, y-1)
        elif action == Action.MOVE_RIGHT:
            return (x, y+1)
        return self.position

    def _is_valid_position(self, position: Tuple[int, int]) -> bool:
        """Check if position is within bounds and not an obstacle"""
        x, y = position
        return (0 <= x < self.room_size[0] and
                0 <= y < self.room_size[1] and
                self.room[x][y] != CellState.OBSTACLE)

    def run(self, max_steps: int = 100):
        """Run the cleaning agent"""
        print("Starting cleaning task...")
        self.display_room()

        for step in range(max_steps):
            print(f"\n--- Step {step + 1} ---")

            if self._goal_test():
                print("Goal achieved! Room is clean!")
                break

            # Agent decision cycle: Perceive -> Think -> Act
            plan = self._formulate_plan()

            if plan:
                action = plan[0]  # Execute first action in plan
                print(f"Agent chooses: {action}")
                self.execute_action(action)
            else:
                print("No plan found, idling...")
                self.execute_action(Action.IDLE)

            self.display_room()
            print(f"Performance: {self.performance}")
            time.sleep(0.5)  # Slow down for demonstration

        print(f"\nFinal Performance: {self.performance}")
        print(f"Total actions taken: {self.actions_taken}")
        print(f"Cells cleaned: {len(self.cleaned_cells)}")

    def display_room(self):
        """Display the current state of the room"""
        print("\nRoom State:")
        for i in range(self.room_size[0]):
            row = ""
            for j in range(self.room_size[1]):
                if (i, j) == self.position:
                    row += "A "  # Agent
                elif self.room[i][j] == CellState.DIRTY:
                    row += "D "  # Dirty
                elif self.room[i][j] == CellState.OBSTACLE:
                    row += "X "  # Obstacle
                else:
                    row += ". "  # Clean
            print(row)

# Demonstration
def demo_cleaning_agent():
    """Demonstrate the cleaning agent"""
    print("=== Goal-based Cleaning Agent Demonstration ===\n")

    # Create and run the agent
    agent = CleaningAgent(room_size=(4, 4), dirt_prob=0.4)
    agent.run(max_steps=50)

if __name__ == "__main__":
    demo_cleaning_agent()

=== Goal-based Cleaning Agent Demonstration ===

Starting cleaning task...

Room State:
A . D . 
. . . D 
D D . . 
. . D D 

--- Step 1 ---
Agent chooses: Action.MOVE_RIGHT
Moved to (0, 1)

Room State:
. A D . 
. . . D 
D D . . 
. . D D 
Performance: -1

--- Step 2 ---
Agent chooses: Action.MOVE_RIGHT
Moved to (0, 2)

Room State:
. . A . 
. . . D 
D D . . 
. . D D 
Performance: -2

--- Step 3 ---
Agent chooses: Action.CLEAN
Cleaned cell (0, 2)

Room State:
. . A . 
. . . D 
D D . . 
. . D D 
Performance: 8

--- Step 4 ---
Agent chooses: Action.MOVE_DOWN
Moved to (1, 2)

Room State:
. . . . 
. . A D 
D D . . 
. . D D 
Performance: 7

--- Step 5 ---
Agent chooses: Action.MOVE_RIGHT
Moved to (1, 3)

Room State:
. . . . 
. . . A 
D D . . 
. . D D 
Performance: 6

--- Step 6 ---
Agent chooses: Action.CLEAN
Cleaned cell (1, 3)

Room State:
. . . . 
. . . A 
D D . . 
. . D D 
Performance: 16

--- Step 7 ---
Agent chooses: Action.MOVE_DOWN
Moved to (2, 3)

Room State:
. . . . 
. . . . 
D D . A